In [1]:
import os 
import sys 
import warnings 

import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 

In [2]:
train_df = pd.read_csv('../../../data/data/train.csv')
val_df = pd.read_csv('../../../data/data/validation.csv')
test_df = pd.read_csv('../../../data/data/test.csv')

In [3]:
print(train_df.columns)
train_df.head()

Index(['Reservation-id', 'Gender', 'Age', 'Ethnicity', 'Educational_Level',
       'Income', 'Country_region', 'Hotel_Type', 'Expected_checkin',
       'Expected_checkout', 'Booking_date', 'Adults', 'Children', 'Babies',
       'Meal_Type', 'Visted_Previously', 'Previous_Cancellations',
       'Deposit_type', 'Booking_channel', 'Required_Car_Parking',
       'Reservation_Status', 'Use_Promotion', 'Discount_Rate', 'Room_Rate'],
      dtype='object')


,Reservation-id,Gender,Age,Ethnicity,Educational_Level,Income,Country_region,Hotel_Type,Expected_checkin,Expected_checkout,...,Meal_Type,Visted_Previously,Previous_Cancellations,Deposit_type,Booking_channel,Required_Car_Parking,Reservation_Status,Use_Promotion,Discount_Rate,Room_Rate
0,39428300,F,40,Latino,Grad,<25K,North,City Hotel,7/1/2015,7/2/2015,...,BB,No,No,No Deposit,Online,Yes,Check-In,Yes,10,218
1,77491756,F,49,Latino,Mid-School,50K -- 100K,East,City Hotel,7/1/2015,7/2/2015,...,BB,No,No,Refundable,Online,Yes,Check-In,No,0,185
2,73747291,F,42,caucasian,Grad,<25K,East,City Hotel,7/2/2015,7/6/2015,...,BB,No,No,No Deposit,Online,Yes,Check-In,No,0,119
3,67301739,M,25,African American,College,>100K,South,Airport Hotels,7/2/2015,7/3/2015,...,BB,No,No,Refundable,Agent,Yes,Check-In,Yes,5,144
4,77222321,F,62,Latino,High-School,25K --50K,East,Resort,7/3/2015,7/4/2015,...,BB,No,No,No Deposit,Direct,No,Check-In,Yes,10,242


column masking

In [4]:
col_mask = ['Gender', 'Age', 'Ethnicity', 'Educational_Level',
       'Income', 'Country_region', 'Hotel_Type', 'Adults', 'Children', 'Babies',
       'Meal_Type', 'Visted_Previously', 'Previous_Cancellations',
       'Deposit_type', 'Booking_channel', 'Required_Car_Parking',
       'Use_Promotion', 'Discount_Rate', 'Room_Rate', 'Reservation_Status']


In [5]:
ms_train_df = train_df.loc[:, col_mask]
ms_val_df = val_df.loc[:, col_mask]

In [23]:
ms_train_df['Reservation_Status'].unique()

array(['Check-In', 'Canceled', 'No-Show'], dtype=object)

categorical encoding

In [6]:
def cat_to_int(df, columns, enc={}):
    df = df.copy()
    if enc == {} or len(columns) > len(enc) :
        maps = enc
        for col in columns:
            if col not in maps:
                mapping = {k: i for i,k in enumerate(df.loc[:,col].unique())}
                maps[col] = mapping
            df[col] = df[col].map(maps[col])
        return df, maps
    else:
        maps = enc
        for col in columns:
            df[col] = df[col].map(maps[col])
        return df

In [9]:
cols = ['Gender','Ethnicity', 'Educational_Level',
       'Income', 'Country_region', 'Hotel_Type','Meal_Type', 'Visted_Previously', 'Previous_Cancellations',
       'Deposit_type', 'Booking_channel', 'Required_Car_Parking', 'Use_Promotion', 'Discount_Rate', 'Reservation_Status']
enc_train_df, maps= cat_to_int(ms_train_df, cols, {'Reservation_Status' : {'Check-In' : 1, 'Canceled' : 2, 'No-Show': 3},})
enc_val_df = cat_to_int(ms_val_df, cols, maps)

In [10]:
enc_train_df.head()

,Gender,Age,Ethnicity,Educational_Level,Income,Country_region,Hotel_Type,Adults,Children,Babies,Meal_Type,Visted_Previously,Previous_Cancellations,Deposit_type,Booking_channel,Required_Car_Parking,Use_Promotion,Discount_Rate,Room_Rate,Reservation_Status
0,F,40,Latino,Grad,<25K,North,City Hotel,2,2,0,0,0,0,0,0,0,0,0,218,1
1,F,49,Latino,Mid-School,50K -- 100K,East,City Hotel,3,3,0,0,0,0,1,0,0,1,1,185,1
2,F,42,caucasian,Grad,<25K,East,City Hotel,3,3,0,0,0,0,0,0,0,1,1,119,1
3,M,25,African American,College,>100K,South,Airport Hotels,4,3,0,0,0,0,1,1,0,0,2,144,1
4,F,62,Latino,High-School,25K --50K,East,Resort,1,1,0,0,0,0,0,2,1,0,0,242,1


In [16]:
def woe_enc(df, columns, target, enc={}):
    df = df.copy()
    maps = enc 
    if maps == {}:
        for col in columns:
            tmp = pd.DataFrame(df.groupby([col])[target].mean())
            tmp['non-target'] = 1 - tmp[target]
            tmp['ratio'] = tmp[target]/tmp['non-target']
            mapping = tmp['ratio'].to_dict()
            maps[col] = mapping
            df[col] = df[col].map(maps[col])
            
        return df, maps
    else:
        for col in columns:
            df[col] = df[col].map(maps[col])
            maps[col] = mapping
        return df
    

In [18]:
import category_encoders as ce 

In [24]:
columns = ['Gender','Ethnicity', 'Educational_Level',
       'Income', 'Country_region', 'Hotel_Type','Meal_Type', 'Visted_Previously', 'Previous_Cancellations']
woe_encoder = ce.CatBoostEncoder(cols=columns)
woe_encoded_train = woe_encoder.fit_transform(enc_train_df[columns], enc_train_df['Reservation_Status']).add_suffix('_woe')
train_features = ms_train_df.join(woe_encoded_train)

woe_encoded_cols = woe_encoded_train.columns

f:\JetBrain Project Files\Pycharm\datastorm-2021\venv\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [25]:
train_features.head()

,Gender,Age,Ethnicity,Educational_Level,Income,Country_region,Hotel_Type,Adults,Children,Babies,...,Reservation_Status,Gender_woe,Ethnicity_woe,Educational_Level_woe,Income_woe,Country_region_woe,Hotel_Type_woe,Meal_Type_woe,Visted_Previously_woe,Previous_Cancellations_woe
0,F,40,Latino,Grad,<25K,North,City Hotel,2,2,0,...,Check-In,1.304884,1.304884,1.304884,1.304884,1.304884,1.304884,1.304884,1.304884,1.304884
1,F,49,Latino,Mid-School,50K -- 100K,East,City Hotel,3,3,0,...,Check-In,1.152442,1.152442,1.304884,1.304884,1.304884,1.152442,1.152442,1.152442,1.152442
2,F,42,caucasian,Grad,<25K,East,City Hotel,3,3,0,...,Check-In,1.101628,1.304884,1.152442,1.152442,1.152442,1.101628,1.101628,1.101628,1.101628
3,M,25,African American,College,>100K,South,Airport Hotels,4,3,0,...,Check-In,1.304884,1.304884,1.304884,1.304884,1.304884,1.304884,1.076221,1.076221,1.076221
4,F,62,Latino,High-School,25K --50K,East,Resort,1,1,0,...,Check-In,1.076221,1.101628,1.304884,1.304884,1.101628,1.304884,1.060977,1.060977,1.060977


In [17]:
maps

{'Reservation_Status': {'check-in': 1, 'cancel': 2, 'no-show': 3},
 'Gender': {'F': 0, 'M': 1},
 'Ethnicity': {'Latino': 0,
  'caucasian': 1,
  'African American': 2,
  'Asian American': 3},
 'Educational_Level': {'Grad': 0,
  'Mid-School': 1,
  'College': 2,
  'High-School': 3},
 'Income': {'<25K': 0, '50K -- 100K': 1, '>100K': 2, '25K --50K': 3},
 'Country_region': {'North': 0, 'East': 1, 'South': 2, 'West': 3},
 'Hotel_Type': {'City Hotel': 0, 'Airport Hotels': 1, 'Resort': 2},
 'Meal_Type': {'BB': 0, 'FB': 1, 'HB': 2},
 'Visted_Previously': {'No': 0, 'Yes': 1},
 'Previous_Cancellations': {'No': 0, 'Yes': 1},
 'Deposit_type': {'No Deposit': 0, 'Refundable': 1, 'Non-Refundable': 2},
 'Booking_channel': {'Online': 0, 'Agent': 1, 'Direct': 2},
 'Required_Car_Parking': {'Yes': 0, 'No': 1},
 'Use_Promotion': {'Yes': 0, 'No': 1},
 'Discount_Rate': {10: 0, 0: 1, 5: 2, 25: 3, 20: 4, 15: 5, 30: 6, 40: 7}}

Lasso feature selection

In [36]:
import matplotlib.pyplot as plt
import numpy as np

from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LassoCV
from sklearn.ensemble import RandomForestClassifier

In [43]:
def Lasso_fs(estimator, X, y, threshold=0.25):
    sfm = SelectFromModel(estimator, threshold=threshold, prefit=False, norm_order=1, max_features=None)
    sfm.fit(X, y)

    feature_idx = sfm.get_support()
    feature_name = X.columns[feature_idx]
    feature_name

    # n_features = sfm.transform(X).shape[1]
    # n_features
    return feature_name

In [45]:
estimator = RandomForestClassifier()
features = Lasso_fs(estimator, enc_train_df.iloc[:, :-1], enc_train_df.iloc[:, -1], 0.1)
print(features)

Index(['Age', 'Room_Rate'], dtype='object')


Recursive feature elimination

In [59]:
from sklearn.feature_selection import RFE
from sklearn.feature_selection import RFECV

In [60]:
def rfe(model, X, y):
    names=pd.DataFrame(X.columns)


    #This is to select 5 variables: can be changed and checked in model for accuracy
    rfe_mod = RFE(estimator=model, n_features_to_select=10, step=1) #RFECV(lin_reg, step=1, cv=5) 
    myvalues=rfe_mod.fit(X,y) #to fit
    myvalues.support_#The mask of selected features.
    myvalues.ranking_ #The feature ranking, such that ranking_[i] corresponds to the ranking position of the i-th feature. Selected (i.e., estimated best) features are assigned rank 1.

    rankings=pd.DataFrame(myvalues.ranking_) #Make it into data frame
    return rankings

In [62]:
fs_mask = rfe(estimator, enc_train_df.iloc[:, :-1], enc_train_df.iloc[:, -1])

In [63]:
fs_mask.reset_index(inplace=True)

In [66]:
ch_cols = fs_mask.loc[fs_mask.iloc[:,1]==1]['index'].values

In [67]:
fin_df = enc_train_df.iloc[:, ch_cols]

In [68]:
fin_df.head()

,Age,Ethnicity,Educational_Level,Income,Country_region,Hotel_Type,Adults,Booking_channel,Discount_Rate,Room_Rate
0,40,0,0,0,0,0,2,0,0,218
1,49,0,1,1,1,0,3,0,1,185
2,42,1,0,0,1,0,3,0,1,119
3,25,2,2,2,2,1,4,1,2,144
4,62,0,3,3,1,2,1,2,0,242


In [4]:
import os
import sys

from src.scripts.test import func

In [1]:
dic = {'awd': 1, 'awc': 2}
len(dic)

2